# Notebook de Test et Comparaison de LLM

## Objectif
Ce notebook permet de tester et comparer des modèles de langage (LLM) de ~1B de paramètres, optimisés pour Google Colab Free.

Pourquoi ce notebook :
- Comprendre le comportement des LLM avant de se lancer dans la compression
- Établir des métriques de base (baseline) pour comparer les modèles
- Apprendre les différentes méthodes d'évaluation des LLM

Tests inclus :
1. Génération de texte simple
2. Conversation avec prompt système
3. Benchmark de performance (vitesse)
4. Perplexité sur WikiText-2
5. Mesure de l'utilisation mémoire
6. Tests de raisonnement basique
7. Latence du premier token (TTFT)

## Installation des dépendances

Installation des bibliothèques nécessaires :
- `transformers` : pour charger et utiliser les modèles
- `torch` : framework de deep learning
- `datasets` : pour accéder à WikiText-2
- `accelerate` : pour optimiser le chargement des modèles

In [ ]:
!pip install -q transformers torch datasets accelerate

## Import des bibliothèques

Import des outils nécessaires pour les tests.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import time
import numpy as np
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device utilisé : {device}")
if device == "cuda":
    print(f"GPU : {torch.cuda.get_device_name(0)}")
    print(f"Mémoire disponible : {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

🖥️ Device utilisé : cuda
GPU : Tesla T4
Mémoire disponible : 15.83 GB


## Configuration des modèles à tester

Liste de modèles recommandés (~1B paramètres) pour Colab Free :

1. **TinyLlama-1.1B** : Version compacte de Llama, très rapide
2. **Qwen2.5-1.5B** : Modèle multilingue de Alibaba
3. **SmolLM2-1.7B** : Modèle performant de HuggingFace
4. **Phi-1.5** : Modèle de Microsoft, bon en raisonnement

Pour changer de modèle, modifiez l'ID du modèle dans la liste ci-dessous.

In [ ]:
# Liste des modèles à tester
MODELS_TO_TEST = [
    # "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    # "Qwen/Qwen2.5-1.5B-Instruct",
    # "HuggingFaceTB/SmolLM2-1.7B-Instruct",
    # "microsoft/phi-1_5",
    "mistralai/Mistral-7B-v0.1",
]

current_model_id = MODELS_TO_TEST[0]
print(f"Modèle sélectionné : {current_model_id}")

📦 Modèle sélectionné : mistralai/Mistral-7B-v0.1


## Chargement du modèle et du tokenizer

**Tokenizer** : Outil qui convertit le texte en tokens (nombres) que le modèle peut traiter.

**AutoModelForCausalLM** : Classe qui charge automatiquement un modèle de langage génératif (prédiction du mot suivant).

Options de chargement :
- `torch_dtype=torch.float16` : Réduit l'utilisation de la mémoire
- `device_map="auto"` : Place automatiquement le modèle sur le GPU si disponible

In [ ]:
print(f"Chargement du modèle {current_model_id}...")
start_time = time.time()

tokenizer = AutoTokenizer.from_pretrained(current_model_id)

model = AutoModelForCausalLM.from_pretrained(
    current_model_id,
    dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

load_time = time.time() - start_time
print(f"Modèle chargé en {load_time:.2f} secondes")

num_params = sum(p.numel() for p in model.parameters())
print(f"Nombre de paramètres : {num_params / 1e9:.2f}B")

⏳ Chargement du modèle mistralai/Mistral-7B-v0.1...


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

✅ Modèle chargé en 329.17 secondes
📊 Nombre de paramètres : 7.24B


---
# TEST 1 : Génération de texte simple

## Objectif
Tester la capacité du modèle à compléter un prompt de manière cohérente.

## Fonctionnement
1. On donne un début de phrase au modèle
2. Le modèle génère la suite mot par mot
3. On observe la qualité et la cohérence du texte généré

## Paramètres de génération :
- `max_new_tokens` : Nombre maximum de mots à générer
- `temperature` : Contrôle la créativité (0.7 = équilibré)
- `top_p` : Échantillonnage nucleus (0.9 = variété)
- `do_sample` : Active l'échantillonnage (plus créatif)

In [ ]:
def test_generation_simple(prompt: str, max_tokens: int = 100):
    """Teste la génération de texte simple."""
    print("="*70)
    print("TEST DE GÉNÉRATION SIMPLE")
    print("="*70)
    print(f"\nPrompt : {prompt}\n")

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    start_time = time.time()

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generation_time = time.time() - start_time

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"Réponse du modèle :\n{generated_text}\n")
    print(f"Temps de génération : {generation_time:.2f}s")
    print(f"Vitesse : {max_tokens/generation_time:.2f} tokens/s")
    print("="*70)

    return generated_text, generation_time

### Exécution du test avec différents prompts

Testez plusieurs types de prompts pour voir comment le modèle se comporte.

In [ ]:
# Exemple 1 : Complétion créative
test_generation_simple(
    "Once upon a time, in a faraway kingdom,",
    max_tokens=80
)

📝 TEST DE GÉNÉRATION SIMPLE

💭 Prompt : Once upon a time, in a faraway kingdom,

🤖 Réponse du modèle :
Once upon a time, in a faraway kingdom, there was a beautiful princess who had a problem.

Every night, when she was supposed to be sleeping, she would wake up and wander around the palace.

She would walk down long hallways, climb up the grand staircase, and even sneak into the kitchen to taste the food.

But no matter how much she tried, she just couldn’t sleep.

⏱️ Temps de génération : 27.94s
🚀 Vitesse : 2.86 tokens/s


('Once upon a time, in a faraway kingdom, there was a beautiful princess who had a problem.\n\nEvery night, when she was supposed to be sleeping, she would wake up and wander around the palace.\n\nShe would walk down long hallways, climb up the grand staircase, and even sneak into the kitchen to taste the food.\n\nBut no matter how much she tried, she just couldn’t sleep.',
 27.935600996017456)

In [ ]:
# Exemple 2 : Explication technique
test_generation_simple(
    "L'intelligence artificielle est",
    max_tokens=100
)

📝 TEST DE GÉNÉRATION SIMPLE

💭 Prompt : L'intelligence artificielle est

🤖 Réponse du modèle :
L'intelligence artificielle est l'une des technologies les plus prometteuses de notre temps. Elle a déjà fait son apparition dans plusieurs domaines, tels que le commerce électronique, la santé et l'éducation.

Mais elle peut aussi être utilisée pour des fins plus malveillantes, telles que la propagande ou la manipulation de l'opinion publique. C'est pourquoi il est important de comprendre comment fonction

⏱️ Temps de génération : 33.16s
🚀 Vitesse : 3.02 tokens/s


("L'intelligence artificielle est l'une des technologies les plus prometteuses de notre temps. Elle a déjà fait son apparition dans plusieurs domaines, tels que le commerce électronique, la santé et l'éducation.\n\nMais elle peut aussi être utilisée pour des fins plus malveillantes, telles que la propagande ou la manipulation de l'opinion publique. C'est pourquoi il est important de comprendre comment fonction",
 33.16127371788025)

---
# TEST 2 : Conversation avec prompt système

## Objectif
Tester la capacité du modèle à suivre des instructions et à répondre dans un format conversationnel.

## Structure d'une conversation :
1. **System prompt** : Instructions sur le comportement du modèle
2. **User message** : La question ou requête de l'utilisateur
3. **Assistant response** : La réponse générée par le modèle

Le prompt système définit la "personnalité" du modèle, similaire à ChatGPT.

In [ ]:
def test_conversation(system_prompt: str, user_message: str, max_tokens: int = 150):
    """Teste le modèle avec un prompt système et un message utilisateur."""
    print("="*70)
    print("TEST DE CONVERSATION AVEC PROMPT SYSTÈME")
    print("="*70)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]

    print(f"\nSystem : {system_prompt}")
    print(f"User : {user_message}\n")

    if tokenizer.chat_template is None:
        tokenizer.chat_template = (
            "<s>"
            "[INST] "
            "{% if messages[0]['role'] == 'system' %}"
                "{{ messages[0]['content'] | trim }}\n"
            "{% endif %}"
            "{{ messages[-1]['content'] | trim }} [/INST]"
        )

    if hasattr(tokenizer, 'apply_chat_template'):
        prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
    else:
        prompt = f"System: {system_prompt}\n\nUser: {user_message}\n\nAssistant:"

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    start_time = time.time()
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    generation_time = time.time() - start_time

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_response = full_response[len(prompt):].strip()

    print(f"Assistant : {assistant_response}\n")
    print(f"Temps de génération : {generation_time:.2f}s")
    print("="*70)

    return assistant_response, generation_time

### Exemples de conversations

Testez différents rôles et questions pour évaluer la flexibilité du modèle.

In [ ]:
# Exemple 1 : Assistant pédagogique
test_conversation(
    system_prompt="Tu es un assistant pédagogique qui explique des concepts complexes de manière simple et accessible.",
    user_message="Qu'est-ce que la perplexité en machine learning ?",
    max_tokens=120
)

💬 TEST DE CONVERSATION AVEC PROMPT SYSTÈME

⚙️ System : Tu es un assistant pédagogique qui explique des concepts complexes de manière simple et accessible.
👤 User : Qu'est-ce que la perplexité en machine learning ?

🤖 Assistant : a perplexité est une mesure de l'incertitude ou de l'imprévisibilité de la sortie d'un modèle de machine learning.
Elle est calculée en divisant le nombre d'échantillons par le logarithme du nombre d'échantillons.

Pour illustrer cette idée, prenons un exemple où nous avons 100 échantillons et 2 classes.
La perplexité est calculée en divisant 100 par le log

⏱️ Temps de génération : 40.81s


("a perplexité est une mesure de l'incertitude ou de l'imprévisibilité de la sortie d'un modèle de machine learning.\nElle est calculée en divisant le nombre d'échantillons par le logarithme du nombre d'échantillons.\n\nPour illustrer cette idée, prenons un exemple où nous avons 100 échantillons et 2 classes.\nLa perplexité est calculée en divisant 100 par le log",
 40.81255865097046)

In [ ]:
# Exemple 2 : Expert technique
test_conversation(
    system_prompt="You are a helpful AI assistant specialized in computer science.",
    user_message="What are the main differences between GPT and BERT architectures?",
    max_tokens=150
)

💬 TEST DE CONVERSATION AVEC PROMPT SYSTÈME

⚙️ System : You are a helpful AI assistant specialized in computer science.
👤 User : What are the main differences between GPT and BERT architectures?

🤖 Assistant : OUT]

GPT and BERT are both neural network-based language models, but they have different architectures and are used for different tasks.

GPT, or Generative Pre-trained Transformer, is an autoregressive language model that is trained on a large amount of text data and can generate new text based on the input it receives. It uses a transformer architecture that allows it to capture long-range dependencies between words in a sentence.

BERT, or Bidirectional Encoder Representations from Transformers, is a pre-trained language representation model that is used for a variety of natural language processing tasks such as question answering, sentiment analysis, and text classification. It

⏱️ Temps de génération : 49.73s


('OUT]\n\nGPT and BERT are both neural network-based language models, but they have different architectures and are used for different tasks.\n\nGPT, or Generative Pre-trained Transformer, is an autoregressive language model that is trained on a large amount of text data and can generate new text based on the input it receives. It uses a transformer architecture that allows it to capture long-range dependencies between words in a sentence.\n\nBERT, or Bidirectional Encoder Representations from Transformers, is a pre-trained language representation model that is used for a variety of natural language processing tasks such as question answering, sentiment analysis, and text classification. It',
 49.72596621513367)

---
# TEST 3 : Benchmark de performance (vitesse)

## Objectif
Mesurer précisément la vitesse de génération du modèle.

## Métriques importantes :
- **Tokens/seconde** : Vitesse de génération
- **Temps de première réponse** : Latence initiale (time to first token)
- **Temps total** : Durée complète de la génération

Après compression, on pourra comparer si le modèle est devenu plus rapide.

In [ ]:
def benchmark_performance(num_runs: int = 5, tokens_to_generate: int = 100):
    """Effectue un benchmark de performance du modèle."""
    print("="*70)
    print("BENCHMARK DE PERFORMANCE")
    print("="*70)
    print(f"\nConfiguration : {num_runs} exécutions, {tokens_to_generate} tokens chacune\n")

    prompt = "The quick brown fox jumps over the lazy dog."
    times = []
    tokens_per_second = []

    for i in range(num_runs):
        print(f"Run {i+1}/{num_runs}...", end=" ")

        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        start_time = time.time()

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=tokens_to_generate,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        elapsed_time = time.time() - start_time
        times.append(elapsed_time)

        tps = tokens_to_generate / elapsed_time
        tokens_per_second.append(tps)

        print(f"{elapsed_time:.2f}s ({tps:.2f} tokens/s)")

    print(f"\nRÉSULTATS :")
    print(f"  - Temps moyen : {np.mean(times):.3f}s (+/-{np.std(times):.3f}s)")
    print(f"  - Vitesse moyenne : {np.mean(tokens_per_second):.2f} tokens/s")
    print(f"  - Vitesse min : {np.min(tokens_per_second):.2f} tokens/s")
    print(f"  - Vitesse max : {np.max(tokens_per_second):.2f} tokens/s")
    print("="*70)

    return {
        "mean_time": np.mean(times),
        "std_time": np.std(times),
        "mean_tokens_per_sec": np.mean(tokens_per_second),
        "min_tokens_per_sec": np.min(tokens_per_second),
        "max_tokens_per_sec": np.max(tokens_per_second)
    }

### Exécution du benchmark

Lance 5 tests et calcule les statistiques.

In [ ]:
benchmark_results = benchmark_performance(num_runs=5, tokens_to_generate=100)

⏱️ BENCHMARK DE PERFORMANCE

📊 Configuration : 5 exécutions, 100 tokens chacune

🔄 Run 1/5... 33.45s (2.99 tokens/s)
🔄 Run 2/5... 33.14s (3.02 tokens/s)
🔄 Run 3/5... 32.87s (3.04 tokens/s)
🔄 Run 4/5... 32.96s (3.03 tokens/s)
🔄 Run 5/5... 33.01s (3.03 tokens/s)

📈 RÉSULTATS :
  • Temps moyen : 33.087s (±0.202s)
  • Vitesse moyenne : 3.02 tokens/s
  • Vitesse min : 2.99 tokens/s
  • Vitesse max : 3.04 tokens/s


---
# TEST 4 : Calcul de la perplexité

## Qu'est-ce que la perplexité ?
La perplexité mesure à quel point le modèle est "surpris" par le texte qu'il voit.

- **Perplexité basse** (ex: 10-20) = Le modèle prédit bien le texte
- **Perplexité haute** (ex: 100+) = Le modèle est confus

## Formule mathématique :
$$\text{Perplexity} = \exp\left(\frac{1}{N} \sum_{i=1}^{N} -\log P(w_i | w_1, ..., w_{i-1})\right)$$

En termes simples : c'est l'exponentielle de la perte (loss) moyenne.

## Dataset : WikiText-2
Un dataset standard pour évaluer les modèles de langage, extrait de Wikipedia.

In [ ]:
def calculate_perplexity(num_samples: int = 100, max_length: int = 512):
    """Calcule la perplexité du modèle sur un échantillon de WikiText-2."""
    print("="*70)
    print("CALCUL DE LA PERPLEXITÉ SUR WIKITEXT-2")
    print("="*70)

    print("\nChargement du dataset WikiText-2...")
    dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")

    texts = [text for text in dataset["text"] if len(text.strip()) > 50]
    texts = texts[:num_samples]

    print(f"{len(texts)} échantillons chargés\n")
    print("Calcul en cours...")

    total_loss = 0
    total_tokens = 0

    model.eval()

    with torch.no_grad():
        for i, text in enumerate(texts):
            if (i + 1) % 20 == 0:
                print(f"  - Progression : {i+1}/{len(texts)} échantillons")

            encodings = tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                max_length=max_length
            ).to(device)

            outputs = model(**encodings, labels=encodings["input_ids"])
            loss = outputs.loss

            num_tokens = encodings["input_ids"].size(1)
            total_loss += loss.item() * num_tokens
            total_tokens += num_tokens

    mean_loss = total_loss / total_tokens
    perplexity = np.exp(mean_loss)

    print(f"\nRÉSULTATS :")
    print(f"  - Perte moyenne (loss) : {mean_loss:.4f}")
    print(f"  - Perplexité : {perplexity:.2f}")
    print(f"  - Tokens analysés : {total_tokens:,}")
    print(f"\nInterprétation :")
    if perplexity < 20:
        print("  Excellente perplexité. Le modèle prédit très bien le texte.")
    elif perplexity < 50:
        print("  Bonne perplexité. Le modèle est performant.")
    elif perplexity < 100:
        print("  Perplexité moyenne. Le modèle est correct mais pourrait être amélioré.")
    else:
        print("  Perplexité élevée. Le modèle a des difficultés avec ce texte.")

    print("="*70)

    return {
        "perplexity": perplexity,
        "mean_loss": mean_loss,
        "total_tokens": total_tokens
    }

### Exécution du test de perplexité

Note : Ce test prend quelques minutes car il analyse 100 échantillons de texte.

In [ ]:
perplexity_results = calculate_perplexity(num_samples=100, max_length=512)

📊 CALCUL DE LA PERPLEXITÉ SUR WIKITEXT-2

📥 Chargement du dataset WikiText-2...


README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

✅ 100 échantillons chargés

🔄 Calcul en cours...
  • Progression : 20/100 échantillons
  • Progression : 40/100 échantillons
  • Progression : 60/100 échantillons
  • Progression : 80/100 échantillons
  • Progression : 100/100 échantillons

📈 RÉSULTATS :
  • Perte moyenne (loss) : 2.1729
  • Perplexité : 8.78
  • Tokens analysés : 19,122

💡 Interprétation :
  ✅ Excellente perplexité ! Le modèle prédit très bien le texte.


### Exécution de la mesure de latence

In [ ]:
def measure_ttft(num_runs: int = 10):
    """Mesure le Time To First Token (TTFT) - latence initiale."""
    print("="*70)
    print("MESURE DE LA LATENCE (TIME TO FIRST TOKEN)")
    print("="*70)
    print(f"\nConfiguration : {num_runs} mesures\n")

    prompt = "The meaning of life is"
    ttft_times = []

    from transformers import TextStreamer

    for i in range(num_runs):
        print(f"Mesure {i+1}/{num_runs}...", end=" ")

        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        start_time = time.time()

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=1,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        ttft = time.time() - start_time
        ttft_times.append(ttft)

        print(f"{ttft*1000:.1f}ms")

    mean_ttft = np.mean(ttft_times)
    std_ttft = np.std(ttft_times)
    min_ttft = np.min(ttft_times)
    max_ttft = np.max(ttft_times)

    print(f"\nRÉSULTATS :")
    print(f"  - TTFT moyen : {mean_ttft*1000:.1f}ms (+/-{std_ttft*1000:.1f}ms)")
    print(f"  - TTFT min : {min_ttft*1000:.1f}ms")
    print(f"  - TTFT max : {max_ttft*1000:.1f}ms")

    print(f"\nInterprétation :")
    if mean_ttft < 0.1:
        print("  Excellente latence (< 100ms)")
    elif mean_ttft < 0.5:
        print("  Bonne latence pour un chatbot.")
    elif mean_ttft < 1.0:
        print("  Latence acceptable mais perceptible.")
    else:
        print("  Latence élevée, l'utilisateur pourrait la remarquer.")

    print("="*70)

    return {
        "mean_ttft_ms": mean_ttft * 1000,
        "std_ttft_ms": std_ttft * 1000,
        "min_ttft_ms": min_ttft * 1000,
        "max_ttft_ms": max_ttft * 1000
    }

In [ ]:
ttft_results = measure_ttft(num_runs=10)

⚡ MESURE DE LA LATENCE (TIME TO FIRST TOKEN)

📊 Configuration : 10 mesures

🔄 Mesure 1/10... 343.1ms
🔄 Mesure 2/10... 332.9ms
🔄 Mesure 3/10... 331.3ms
🔄 Mesure 4/10... 327.8ms
🔄 Mesure 5/10... 338.0ms
🔄 Mesure 6/10... 333.2ms
🔄 Mesure 7/10... 333.2ms
🔄 Mesure 8/10... 334.1ms
🔄 Mesure 9/10... 358.8ms
🔄 Mesure 10/10... 378.9ms

📈 RÉSULTATS :
  • TTFT moyen : 341.1ms (±15.1ms)
  • TTFT min : 327.8ms
  • TTFT max : 378.9ms

💡 Interprétation :
  ✅ Bonne latence pour un chatbot.


---
# TEST 7 : Latence du premier token (TTFT)

## Objectif
Mesurer le temps avant la première réponse du modèle.

## Qu'est-ce que le TTFT ?
**Time To First Token** = Le délai entre l'envoi du prompt et la génération du premier mot.

## Pourquoi c'est important :
- **Expérience utilisateur** : Un TTFT bas = chatbot réactif
- **Compression** : Peut réduire la latence (modèle plus petit = chargement plus rapide)
- **Production** : Métrique clé pour les applications en temps réel

## Différence avec le benchmark de vitesse :
- **TTFT** : Mesure la latence initiale
- **Tokens/sec** : Mesure la vitesse de génération continue

### Exécution des tests de raisonnement

In [ ]:
def test_basic_reasoning():
    """Teste les capacités de raisonnement basique du modèle."""
    print("="*70)
    print("TEST DE RAISONNEMENT BASIQUE")
    print("="*70)

    test_cases = [
        {
            "category": "Arithmétique",
            "question": "What is 2 + 2? Answer with just the number.",
            "expected_keywords": ["4", "four"],
            "max_tokens": 10
        },
        {
            "category": "Arithmétique",
            "question": "Calculate: 15 - 7 = ? Give only the result.",
            "expected_keywords": ["8", "eight"],
            "max_tokens": 10
        },
        {
            "category": "Logique",
            "question": "True or False: A cat is an animal.",
            "expected_keywords": ["true", "yes"],
            "max_tokens": 10
        },
        {
            "category": "Logique",
            "question": "Is the sky typically blue? Answer Yes or No.",
            "expected_keywords": ["yes", "true"],
            "max_tokens": 10
        },
        {
            "category": "Classification",
            "question": "Which of these is a fruit: car, apple, or table? Answer in one word.",
            "expected_keywords": ["apple"],
            "max_tokens": 10
        },
        {
            "category": "Compréhension",
            "question": "What is the capital of France? One word answer.",
            "expected_keywords": ["paris"],
            "max_tokens": 10
        }
    ]

    results = []
    correct_count = 0

    for i, test in enumerate(test_cases):
        print(f"\n{'='*70}")
        print(f"Question {i+1}/{len(test_cases)} - {test['category']}")
        print(f"{'='*70}")
        print(f"Question: {test['question']}")

        inputs = tokenizer(test['question'], return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=test['max_tokens'],
                temperature=0.1,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )

        full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        answer = full_response[len(test['question']):].strip().lower()

        is_correct = any(keyword in answer for keyword in test['expected_keywords'])

        print(f"Réponse : {answer[:100]}")
        print(f"Attendu : {', '.join(test['expected_keywords'])}")

        if is_correct:
            print(f"Correct")
            correct_count += 1
        else:
            print(f"Incorrect")

        results.append({
            "category": test['category'],
            "question": test['question'],
            "answer": answer,
            "expected": test['expected_keywords'],
            "correct": is_correct
        })

    print(f"\n{'='*70}")
    print(f"RÉSUMÉ DES RÉSULTATS")
    print(f"{'='*70}")
    print(f"  - Score : {correct_count}/{len(test_cases)} ({(correct_count/len(test_cases))*100:.1f}%)")

    categories = {}
    for result in results:
        cat = result['category']
        if cat not in categories:
            categories[cat] = {'correct': 0, 'total': 0}
        categories[cat]['total'] += 1
        if result['correct']:
            categories[cat]['correct'] += 1

    print(f"\n  - Par catégorie :")
    for cat, stats in categories.items():
        score = (stats['correct']/stats['total'])*100
        print(f"    - {cat} : {stats['correct']}/{stats['total']} ({score:.0f}%)")

    print("="*70)

    return {
        "score": correct_count,
        "total": len(test_cases),
        "percentage": (correct_count/len(test_cases))*100,
        "details": results
    }

In [ ]:
reasoning_results = test_basic_reasoning()

🧠 TEST DE RAISONNEMENT BASIQUE

Question 1/6 - Arithmétique
❓ What is 2 + 2? Answer with just the number.
🤖 Réponse : what is 2 + 2?
✅ Attendu : 4, four
❌ Incorrect

Question 2/6 - Arithmétique
❓ Calculate: 15 - 7 = ? Give only the result.
🤖 Réponse : ### guidance

the distribut
✅ Attendu : 8, eight
❌ Incorrect

Question 3/6 - Logique
❓ True or False: A cat is an animal.
🤖 Réponse : true or false: a cat is a
✅ Attendu : true, yes
✅ Correct !

Question 4/6 - Logique
❓ Is the sky typically blue? Answer Yes or No.
🤖 Réponse : the sky is blue because of the way
✅ Attendu : yes, true
❌ Incorrect

Question 5/6 - Classification
❓ Which of these is a fruit: car, apple, or table? Answer in one word.
🤖 Réponse : the answer is apple.

the
✅ Attendu : apple
✅ Correct !

Question 6/6 - Compréhension
❓ What is the capital of France? One word answer.
🤖 Réponse : paris.

what is the capital of france
✅ Attendu : paris
✅ Correct !

📊 RÉSUMÉ DES RÉSULTATS
  • Score : 3/6 (50.0%)

  • Par catégorie :
   

---
# TEST 6 : Tests de raisonnement basique

## Objectif
Évaluer les capacités de raisonnement élémentaire du modèle.

## Pourquoi c'est important :
- Vérifie que la compression ne détruit pas les capacités de base
- Plus objectif que la génération créative (réponses vérifiables)
- Facile à comparer : bonne réponse ou mauvaise réponse

## Types de tests :
1. **Arithmétique simple** : Addition, soustraction
2. **Logique de base** : Vrai/Faux, classification
3. **Compréhension** : Questions factuelles
4. **Cohérence** : Réponses contradictoires ou non

### Exécution de la mesure mémoire

In [ ]:
def measure_memory_usage():
    """Mesure l'utilisation de la mémoire GPU et la taille du modèle."""
    print("="*70)
    print("MESURE DE L'UTILISATION MÉMOIRE")
    print("="*70)

    model_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
    model_size_gb = model_size_bytes / (1024**3)

    buffer_size_bytes = sum(b.numel() * b.element_size() for b in model.buffers())
    buffer_size_gb = buffer_size_bytes / (1024**3)

    total_size_gb = model_size_gb + buffer_size_gb

    print(f"\nTaille du modèle en mémoire :")
    print(f"  - Paramètres : {model_size_gb:.3f} GB")
    print(f"  - Buffers : {buffer_size_gb:.3f} GB")
    print(f"  - Total : {total_size_gb:.3f} GB")

    if torch.cuda.is_available():
        gpu_allocated = torch.cuda.memory_allocated() / (1024**3)
        gpu_reserved = torch.cuda.memory_reserved() / (1024**3)
        gpu_total = torch.cuda.get_device_properties(0).total_memory / (1024**3)

        print(f"\nUtilisation GPU :")
        print(f"  - Mémoire allouée : {gpu_allocated:.3f} GB")
        print(f"  - Mémoire réservée : {gpu_reserved:.3f} GB")
        print(f"  - Mémoire totale : {gpu_total:.2f} GB")
        print(f"  - Utilisation : {(gpu_allocated/gpu_total)*100:.1f}%")

        if gpu_total < 16:
            if gpu_allocated < 12:
                print(f"\n  Compatible Colab Free (< 12 GB utilisés)")
            else:
                print(f"\n  Risque pour Colab Free (> 12 GB utilisés)")
    else:
        print(f"\nCPU utilisé - Pas de statistiques GPU")

    disk_size_gb = (num_params * 2) / (1024**3)
    print(f"\nTaille estimée sur disque (float16) : {disk_size_gb:.3f} GB")

    print("="*70)

    return {
        "model_size_gb": model_size_gb,
        "total_size_gb": total_size_gb,
        "gpu_allocated_gb": torch.cuda.memory_allocated() / (1024**3) if torch.cuda.is_available() else 0,
        "gpu_reserved_gb": torch.cuda.memory_reserved() / (1024**3) if torch.cuda.is_available() else 0,
        "disk_size_gb": disk_size_gb
    }

In [ ]:
memory_results = measure_memory_usage()

💾 MESURE DE L'UTILISATION MÉMOIRE

📊 Taille du modèle en mémoire :
  • Paramètres : 13.489 GB
  • Buffers : 0.000 GB
  • Total : 13.489 GB

🖥️ Utilisation GPU :
  • Mémoire allouée : 12.440 GB
  • Mémoire réservée : 12.701 GB
  • Mémoire totale : 14.74 GB
  • Utilisation : 84.4%

  ⚠️ Risque pour Colab Free (> 12 GB utilisés)

💿 Taille estimée sur disque (float16) : 13.489 GB


---
# TEST 5 : Mesure de l'utilisation mémoire

## Objectif
Mesurer précisément la mémoire GPU/RAM utilisée par le modèle.

## Pourquoi c'est important pour la compression :
- La compression vise souvent à réduire l'empreinte mémoire
- Permet de voir si le modèle tient dans les 15 GB de Colab Free
- Métrique clé pour comparer avant/après compression

## Métriques mesurées :
- **Mémoire GPU allouée** : Espace utilisé sur la carte graphique
- **Mémoire GPU réservée** : Espace total réservé par PyTorch
- **Taille du modèle** : Poids en mémoire (GB)
- **Taille sur disque** : Espace occupé si sauvegardé

---
# Résumé des résultats

Cette cellule affiche un tableau récapitulatif de tous les tests effectués.

In [ ]:
def display_summary():
    """Affiche un résumé complet des tests effectués."""
    print("\n" + "="*70)
    print("RÉSUMÉ COMPLET DES TESTS")
    print("="*70)
    print(f"\nModèle testé : {current_model_id}")
    print(f"Nombre de paramètres : {num_params / 1e9:.2f}B")
    print(f"Device : {device}")
    print("\n" + "-"*70)

    if 'benchmark_results' in globals():
        print(f"\nPerformance (vitesse) :")
        print(f"  - Vitesse moyenne : {benchmark_results['mean_tokens_per_sec']:.2f} tokens/s")
        print(f"  - Temps de génération : {benchmark_results['mean_time']:.3f}s")

    if 'ttft_results' in globals():
        print(f"\nLatence (TTFT) :")
        print(f"  - Time to First Token : {ttft_results['mean_ttft_ms']:.1f}ms")

    if 'perplexity_results' in globals():
        print(f"\nQualité (perplexité) :")
        print(f"  - Perplexité : {perplexity_results['perplexity']:.2f}")
        print(f"  - Loss moyenne : {perplexity_results['mean_loss']:.4f}")

    if 'reasoning_results' in globals():
        print(f"\nRaisonnement basique :")
        print(f"  - Score : {reasoning_results['score']}/{reasoning_results['total']} ({reasoning_results['percentage']:.1f}%)")

    if 'memory_results' in globals():
        print(f"\nUtilisation mémoire :")
        print(f"  - Taille du modèle : {memory_results['model_size_gb']:.3f} GB")
        print(f"  - GPU alloué : {memory_results['gpu_allocated_gb']:.3f} GB")
        print(f"  - Taille sur disque : {memory_results['disk_size_gb']:.3f} GB")

    print("\n" + "="*70)
    print("\nProchaines étapes :")
    print("  1. Sauvegarder ces résultats comme baseline")
    print("  2. Appliquer des techniques de compression (quantization, pruning, distillation)")
    print("  3. Ré-exécuter ces tests pour comparer les performances")
    print("  4. Analyser le trade-off : vitesse vs qualité vs mémoire")
    print("="*70)

display_summary()


📋 RÉSUMÉ COMPLET DES TESTS

🤖 Modèle testé : mistralai/Mistral-7B-v0.1
📊 Nombre de paramètres : 7.24B
🖥️ Device : cuda

----------------------------------------------------------------------

⏱️ Performance (vitesse) :
  • Vitesse moyenne : 3.02 tokens/s
  • Temps de génération : 33.087s

⚡ Latence (TTFT) :
  • Time to First Token : 341.1ms

📊 Qualité (perplexité) :
  • Perplexité : 8.78
  • Loss moyenne : 2.1729

🧠 Raisonnement basique :
  • Score : 3/6 (50.0%)

💾 Utilisation mémoire :
  • Taille du modèle : 13.489 GB
  • GPU alloué : 12.440 GB
  • Taille sur disque : 13.489 GB


💡 Prochaines étapes :
  1. Sauvegarder ces résultats comme baseline
  2. Appliquer des techniques de compression (quantization, pruning, distillation)
  3. Ré-exécuter ces tests pour comparer les performances
  4. Analyser le trade-off : vitesse vs qualité vs mémoire


---
# Tester un autre modèle

Pour tester un autre modèle, revenez à la cellule "Configuration des modèles" et changez le modèle sélectionné, puis ré-exécutez les cellules suivantes.

---
# Sauvegarde des résultats

Optionnel : Sauvegardez vos résultats dans un fichier JSON pour comparaison future.

In [ ]:
import json
from datetime import datetime

def save_results(filename="llm_test_results.json"):
    """Sauvegarde les résultats des tests dans un fichier JSON."""
    results = {
        "model_id": current_model_id,
        "num_parameters": num_params,
        "device": device,
        "timestamp": datetime.now().isoformat(),
        "benchmark": benchmark_results if 'benchmark_results' in globals() else None,
        "ttft": ttft_results if 'ttft_results' in globals() else None,
        "perplexity": perplexity_results if 'perplexity_results' in globals() else None,
        "reasoning": reasoning_results if 'reasoning_results' in globals() else None,
        "memory": memory_results if 'memory_results' in globals() else None
    }

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"Résultats sauvegardés dans {filename}")

save_results()

✅ Résultats sauvegardés dans llm_test_results.json


---
# Ressources supplémentaires

## Pour aller plus loin :

### Techniques de compression de LLM :
1. **Quantization** : Réduire la précision des poids (float16 vers int8 vers int4)
2. **Pruning** : Supprimer les connexions peu importantes
3. **Knowledge Distillation** : Entraîner un petit modèle à imiter un grand
4. **Low-Rank Factorization** : Décomposer les matrices en matrices plus petites

### Bibliothèques utiles :
- **bitsandbytes** : Quantization facile
- **GPTQ** : Quantization post-training
- **GGML/llama.cpp** : Exécution optimisée sur CPU

### Autres métriques à explorer :
- **BLEU, ROUGE** : Qualité de la génération
- **Benchmarks académiques** : MMLU, HellaSwag, TruthfulQA
- **Taille du modèle** : Mémoire occupée (GB)
- **Énergie consommée** : Impact environnemental